In [12]:
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
import warnings
warnings.filterwarnings("ignore")

In [13]:
TRAIN_DIR = 'images\\train'
TEST_DIR = 'images\\validation'

In [14]:
def createDataFrame(dir:str):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for img_name in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, img_name))
            labels.append(label)
        print("label: {} completed!".format(label))
    return image_paths, labels

In [15]:
train = pd.DataFrame()
test = pd.DataFrame()

train['image'], train['label'] = createDataFrame(TRAIN_DIR)
print()
test['image'], test['label'] = createDataFrame(TEST_DIR)

label: angry completed!
label: disgust completed!
label: fear completed!
label: happy completed!
label: neutral completed!
label: sad completed!
label: surprise completed!

label: angry completed!
label: disgust completed!
label: fear completed!
label: happy completed!
label: neutral completed!
label: sad completed!
label: surprise completed!


In [16]:
os.getcwd()

'D:\\Study\\FPT_Uni\\Summer-2024\\CPV301\\CPV_Assignment\\EmotionalRecognition'

In [17]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features
        

In [18]:
train_features = extract_features(train['image'])

100%|██████████| 28821/28821 [00:04<00:00, 6587.14it/s]


In [19]:
test_features = extract_features(test['image'])

100%|██████████| 7066/7066 [00:01<00:00, 6615.10it/s]


In [20]:
x_train = train_features/255.0
x_test = test_features/255.0

In [21]:
le = LabelEncoder()

le.fit(train['label'])

LabelEncoder()

In [22]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [23]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [24]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7, activation='softmax'))

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 [==============================] - 16s 55ms/step - loss: 1.8132 - accuracy: 0.2440 - val_loss: 1.7348 - val_accuracy: 0.2941
Epoch 2/100
226/226 [==============================] - 12s 52ms/step - loss: 1.6449 - accuracy: 0.3465 - val_loss: 1.5141 - val_accuracy: 0.4164
Epoch 3/100
226/226 [==============================] - 12s 53ms/step - loss: 1.4898 - accuracy: 0.4248 - val_loss: 1.3716 - val_accuracy: 0.4740
Epoch 4/100
226/226 [==============================] - 12s 53ms/step - loss: 1.4018 - accuracy: 0.4608 - val_loss: 1.2892 - val_accuracy: 0.5125
Epoch 5/100
226/226 [==============================] - 12s 53ms/step - loss: 1.3373 - accuracy: 0.4878 - val_loss: 1.2396 - val_accuracy: 0.5250
Epoch 6/100
226/226 [==============================] - 12s 53ms/step - loss: 1.2876 - accuracy: 0.5047 - val_loss: 1.2037 - val_accuracy: 0.5430
Epoch 7/100
226/226 [==============================] - 12s 53ms/step - loss: 1.2525 - accuracy: 0.5211 - val_loss: 1.1921 - val_ac

In [27]:
model_json = model.to_json()
with open("weight\\model.json", "w") as json_file:
    json_file.write(model_json)
model.save("weight\\model_emotional_recognition.h5")

In [35]:
json_file = open('weight\\model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("weight\\model_emotional_recognition.h5")

In [36]:
labels = ['angry', 'disgust', 'fear', 'happy','neutral', 'sad', 'surprise']

In [37]:
def ef(image):
    img = load_img(image, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature/255.0    

In [39]:
image = 'images\\validation\\fear\\21.jpg'
img = ef(image)
pred = model.predict(img)
pred_label = labels[np.argmax(pred)]

print('Original label is {}'.format('fear'))
print('Predicted label is {}'.format(pred_label))

Original label is fear
Predicted label is fear
